## AIPI 590 (Deep Reinforecement Learning) HOMEWORK 1, Question 2

In [ ]:
NAME = "Archit Kaila"
COLLABORATORS = ""

### Question 1.2: Train an agent to play any game/task in SB3 using A2C with Colab. Use the APIs provided in SB3 to run the Google Colab experiments and to train/test the RL agent.
#### Requirements:
    - Pick a task/game in SB3.
    - Add a TensorBoard to visualize the training curves.   
    - Include/record the final evaluation video.
    - Saving/loading the policy/model.
#### Submission:
    - Submit a Colab notebook including the training/testing
    - experiments and results.

##### Install Stable Baseline3 And Other Dependencies

In [ ]:
## Stable Baseline3 Installation
!pip install stable-baselines3[extra] 

## visualization requirment
!apt-get install ffmpeg freeglut3-dev xvfb
!pip install pyglet

In [ ]:
## Library Imports
import gym
import numpy as np
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy

### Creating a GYM environment and instanciating an agent

In [ ]:
## Create a GYM environment of CartPole-v1
environment = gym.make('CartPole-v1')

## initialize tensorboard log directory
tensorboard_logs = "data/tb/"

## Initialize an Agent (Actor Critic)
model = A2C('MlpPolicy', environment, verbose=1, normalize_advantage=True, tensorboard_log=tensorboard_logs)

### Intialize environment for evaluation

In [ ]:
## Create environment for evaluation
evaluation_environment = gym.make('CartPole-v1')

## Setting up agent with randomly before training with #eval episodes = 100
reward_mean, reward_std = evaluate_policy(model, evaluation_environment, n_eval_episodes=100)

print(f'Initial Mean Reward: {np.round(reward_mean,2)} +/- {np.round(reward_std,2)}')

### Agent Training

In [ ]:
## Initialize Train loop for the agent with 500k steps
model.learn(total_timesteps=500000)

### Agent Evaluation

In [ ]:
## Estimating rewards and evaluating our trained agent
reward_mean, reward_std = evaluate_policy(model, evaluation_environment, n_eval_episodes=100)

print(f'Mean Reward: {np.round(reward_mean,2)} +/- {np.round(reward_std,2)}')

### Setup environment to play and record Video inside Colab notebook

In [ ]:
## Setup a dummy display
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

### Function to Record Agent Actions In Video

In [ ]:
## Function to record video of agent
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def video_recorder(environment_id, model, video_length=500, prefix='', video_destination='videos/'):
  ## Creating dummy environment
  evaluation_env = DummyVecEnv([lambda: gym.make(environment_id)])
  
  ## Starting the video at step=0 and recording the next 500 steps
  evaluation_env = VecVideoRecorder(evaluation_env, video_folder=video_destination,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  ## Iterating and executing actions
  obs = evaluation_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = evaluation_env.step(action)

  ## Closing video recorder
  evaluation_env.close()

### Functions To Render and Display Video Frames

In [ ]:
## Function to show video play inside the notebook
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

### Vizualizing Video Recording Of Our Trained Agent

In [ ]:
## Calling the video_recording function to record the agent actions
video_recorder('CartPole-v1', model, video_length=500, prefix='a2c-cartpole')

## Display recorded video inline
show_videos('videos', prefix='a2c')

### Tensorboard Visualization

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./data/tb/